Набор данных, который мы будем использовать для этого питоньего проекта - назовем его news.csv (новостями, так сказать).

У этого набора данных форма 7796×4 - не спрашивайте, почему такая странная, просто примите как данность! Первый столбик - это  ID новости (чтобы каждой новости достался свой номерок), второй и третий - заголовок и сам текст (где новости разглагольствуют о своих делах), а четвертый столбик содержит метки, которые как бы намекают: "Эй, это же настоящая новость или опять какой-то бред?"

Импортируем все нужные библиотеки

In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

Загоняем данные в DataFrame, смотрим, что там по форме и первые 5 записей

In [2]:

df=pd.read_csv("data/news.csv")


df.shape
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


А теперь давай-ка вытащим так нужные нам метки из нашего DataFrame

In [3]:
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [5]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=9)

In [6]:
tokinizer=TfidfVectorizer(stop_words='english', max_df=0.8)
x_tokinizer_train=tokinizer.fit_transform(x_train) 
x_tokinizer_test=tokinizer.transform(x_test)

потестим различные методы

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(x_tokinizer_train, y_train)
y_pred = lr.predict(x_tokinizer_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 91.0%


In [9]:
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[585,  34],
       [ 80, 568]], dtype=int64)

In [11]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')  
svm.fit(x_tokinizer_train, y_train)
y_pred = svm.predict(x_tokinizer_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.13%


In [12]:
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[589,  30],
       [ 57, 591]], dtype=int64)

In [13]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(x_tokinizer_train, y_train)
y_pred = nb.predict(x_tokinizer_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 86.19%


In [16]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, random_state=9)
gb.fit(x_tokinizer_train, y_train)
y_pred = gb.predict(x_tokinizer_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 88.24%


In [18]:

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(x_tokinizer_train,y_train)

y_pred=pac.predict(x_tokinizer_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.53%


In [19]:
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[587,  32],
       [ 50, 598]], dtype=int64)